In [1]:
import open3d as o3
import numpy as np
import os
import sys

sys.path.append('/home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/lib/pyigllib')
import pyigl as igl
import iglhelpers


## Draw functions

In [2]:
def build_spheres(points, colors, radius=1.0):
    points = points.copy()
    spheres = []
    num = points.shape[0]
    for i in range(num):
        ms = o3.create_mesh_sphere(radius)
        ms.compute_vertex_normals()
        ms.paint_uniform_color(colors[i])
        transform_vertices = np.asarray(ms.vertices) + points[i] # Bug exist here!!?
        ms.vertices = o3.Vector3dVector(transform_vertices)
        spheres.append(ms)
    return spheres
        

def buildToothKeypoints13(points):
    points = points.copy()
    assert(points.shape[0] <= 7)
    colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [0.5, 0.5, 0.5]]        
    geos = build_spheres(points, colors[:points.shape[0]], 0.2)
    return geos


def show_mesh_keypoints13(mesh, points): 
    mesh.compute_vertex_normals()
    o3.draw_geometries([mesh] + buildToothKeypoints13(points))

    #pcd = o3.PointCloud()
    #pcd.points = mesh.vertices
    #o3.draw_geometries([pcd] + buildToothKeypoints13(points))
    
def show_mesh_keypoints13_2(vs, fs, points):
    mesh = o3.open3d.TriangleMesh()
    mesh.vertices = o3.Vector3dVector(vs.copy())
    mesh.triangles = o3.Vector3iVector(fs.copy())
    mesh.compute_vertex_normals()
    o3.draw_geometries([mesh] + buildToothKeypoints13(points))
    
def show_cloud_keypoints13(cloud, points):
    pcd = o3.PointCloud()
    pcd.points = o3.Vector3dVector(cloud)
    o3.draw_geometries([pcd] + buildToothKeypoints13(points))
    
    
def compare_mesh_keypoints13(vs, fs, points1, points2):
    mesh = o3.open3d.TriangleMesh()
    mesh.vertices = o3.Vector3dVector(vs.copy())
    mesh.triangles = o3.Vector3iVector(fs.copy())
    mesh.compute_vertex_normals()
    
    colors1 = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [0.5, 0.5, 0.5]]        
    geos1 = build_spheres(points1, colors1[:points1.shape[0]], 0.2)
    colors2 = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [0.5, 0.5, 0.5]]        
    #colors2 = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0.0, 0.0, 0.0]]   
    geos2 = build_spheres(points2, colors2[:points2.shape[0]], 0.3)
    
    o3.draw_geometries([mesh] + geos1 + geos2)

def compare_cloud_keypoints13(vs, fs, points1, points2):
    mesh = o3.PointCloud()
    mesh.points = o3.Vector3dVector(vs)
    
    colors1 = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [0.5, 0.5, 0.5]]        
    geos1 = build_spheres(points1, colors1[:points1.shape[0]], 0.2)
    colors2 = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0], [1, 0, 1], [0, 1, 1], [0.5, 0.5, 0.5]]        
    #colors2 = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0.0, 0.0, 0.0]]   
    geos2 = build_spheres(points2, colors2[:points2.shape[0]], 0.3)
    
    o3.draw_geometries([mesh] + geos1 + geos2)


## 2. Load train and test dataset

In [3]:
def load_names(names_fn):
    with open(names_fn) as f:
        names = [l.rstrip() for l in f.readlines()]
    return names

def load_mesh(obj_fn):
    V = igl.eigen.MatrixXd()
    F = igl.eigen.MatrixXi()
    igl.readOBJ(obj_fn, V, F)
    return iglhelpers.e2p(V), iglhelpers.e2p(F)


In [4]:
data_dir = r'/home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/exp1/split1/'

train_names = load_names(os.path.join(data_dir, 'train_names.txt'))
train_keypoints = np.loadtxt(os.path.join(data_dir, 'train_keypoints.txt'))
train_keypoints = train_keypoints.reshape(train_keypoints.shape[0], -1, 3)
train_refpoints = np.loadtxt(os.path.join(data_dir, 'train_refpoints.txt'))

test_names = load_names(os.path.join(data_dir, 'test_names.txt'))
test_keypoints = np.loadtxt(os.path.join(data_dir, 'test_keypoints.txt'))
test_keypoints = test_keypoints.reshape(test_keypoints.shape[0], -1, 3)
test_refpoints = np.loadtxt(os.path.join(data_dir, 'test_refpoints.txt'))

print('train names: ', len(train_names))
print('test names: ', len(test_names))

print('train samples: ', train_keypoints.shape)
print('test samples: ', test_keypoints.shape)

train names:  42
test names:  11
train samples:  (42, 7, 3)
test samples:  (11, 7, 3)


In [5]:
# Let's show some training samples
idx = 0
mesh_name = train_names[idx]
vs, fs = load_mesh(mesh_name)

print('{}th training sample '.format(idx))
print('name: {}'.format(mesh_name))

print('vs.shape: {}'.format(vs.shape))
print('fs.shape: {}'.format(vs.shape))

# show 7 keypoints
keypoints = train_keypoints[idx]
show_mesh_keypoints13_2(vs, fs, keypoints)

# show center point
print('train_refpoints.shape: ', train_refpoints.shape)
ref_pt = train_refpoints[idx]
print('ref_pt.shape: ', ref_pt.shape)
show_cloud_keypoints13(vs, ref_pt.reshape(-1, 3))

0th training sample 
name: /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/605.obj
vs.shape: (2676, 3)
fs.shape: (2676, 3)
train_refpoints.shape:  (42, 3)
ref_pt.shape:  (3,)


In [6]:
# Let's show some test samples
idx = 0
mesh_name = test_names[idx]
vs, fs = load_mesh(mesh_name)

print('{}th test sample '.format(idx))
print('name: {}'.format(mesh_name))

print('vs.shape: {}'.format(vs.shape))
print('fs.shape: {}'.format(vs.shape))

# show 7 keypoints
keypoints = test_keypoints[idx]
show_mesh_keypoints13_2(vs, fs, keypoints)

# show center point
print('test_refpoints.shape: ', test_refpoints.shape)
ref_pt = test_refpoints[idx]
print('ref_pt.shape: ', ref_pt.shape)

print('ref_pt: ', ref_pt)

show_cloud_keypoints13(vs, ref_pt.reshape(-1, 3))

0th test sample 
name: /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/654.obj
vs.shape: (2300, 3)
fs.shape: (2300, 3)
test_refpoints.shape:  (11, 3)
ref_pt.shape:  (3,)
ref_pt:  [15.1996 10.7677 21.2424]


## 3. Train and test on keypoints subset

### 3.1 Define keypoints subset
Select some types of keypoints to train and test

In [7]:
# tooth 13 keypoints: 
# 1.occc, 2.glp, 3.gbp
# 4.occm, 5.occd
# 6.mrm,  7. mrd


# Active keypoints indices
# These keypoints will be trained and tested
#aki = [0, 5, 6]  # occc, mrm, mrd
aki = [0, 1, 2, 3, 4, 5, 6]  # all keypoints

sub_train_keypoints = train_keypoints[:,aki,:]
sub_test_keypoints = test_keypoints[:,aki,:]

print('sub train keypoints: ', sub_train_keypoints.shape)
print('sub test keypoints: ', sub_test_keypoints.shape)

sub train keypoints:  (42, 7, 3)
sub test keypoints:  (11, 7, 3)


### 3.2 Load testing result

In [8]:
# load fitting and testing result
test_res = np.loadtxt('./test_res.txt')
test_res = test_res.reshape(test_res.shape[0], -1, 3)

print('test result shape: ', test_res.shape)
assert(test_res.shape == sub_test_keypoints.shape)

test result shape:  (11, 7, 3)


In [9]:
idx = 0


keypoints = test_res[idx]
print('keypoints: ', keypoints)


refpoint = test_refpoints[idx]
print('refpoint: ', refpoint)


center = test_res[idx] - refpoint
center



keypoints:  [[16.6314  7.745  22.9924]
 [11.5405 12.1995 18.2197]
 [16.3132 16.3359 22.0379]
 [15.0405  9.3359 24.2651]
 [17.586   8.3813 20.7651]
 [13.4496  9.9722 22.9924]
 [16.3132  9.6541 18.856 ]]
refpoint:  [15.1996 10.7677 21.2424]


array([[ 1.4318, -3.0227,  1.75  ],
       [-3.6591,  1.4318, -3.0227],
       [ 1.1136,  5.5682,  0.7955],
       [-0.1591, -1.4318,  3.0227],
       [ 2.3864, -2.3864, -0.4773],
       [-1.75  , -0.7955,  1.75  ],
       [ 1.1136, -1.1136, -2.3864]])

In [10]:
kp_id = 3

refpoints = test_refpoints
center = test_res[:,kp_id,:] - refpoints
center

array([[-0.1591, -1.4318,  3.0227],
       [-2.0682,  1.4318,  2.3864],
       [-2.0682, -1.1136,  2.0682],
       [-1.1136, -1.4318,  2.7045],
       [-0.7955, -1.4318,  2.7045],
       [-0.1591, -1.75  ,  2.7045],
       [-1.1136, -1.4318,  3.3409],
       [ 0.4773, -2.0682,  2.7045],
       [-0.1591, -1.4318,  3.3409],
       [-1.1136, -1.4318,  3.3409],
       [-1.1136, -1.75  ,  2.7045]])

In [11]:
# compare test result with ground truth

def compare_test_gt(idx):
    assert(idx < test_res.shape[0])
    mesh_name = test_names[idx]
    vs, fs = load_mesh(mesh_name)
    gt = sub_test_keypoints[idx]
    est = test_res[idx]
    #print('keypoints diff: {}'.format(est - gt))
    #compare_cloud_keypoints13(vs, fs, gt, est)
    compare_mesh_keypoints13(vs, fs, gt, est)
    
#indices = np.arange(10)
#indices = (np.random.rand(10) * test_res.shape[0]).astype(np.int32)
#indices = [0 + 12*i for i in range(4)]
indices = np.arange(test_res.shape[0])

for idx in indices:
    print('current idx: ', idx)
    print('name: ', test_names[idx])
    compare_test_gt(idx)

current idx:  0
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/654.obj
current idx:  1
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/655.obj
current idx:  2
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/656.obj
current idx:  3
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/657.obj
current idx:  4
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/658.obj
current idx:  5
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/659.obj
current idx:  6
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/660.obj
current idx:  7
name:  /home/yalong/yalong/project/KeyPointsEstimation/V2V-PoseNet-pytorch/experiments/tooth/dataset1/

In [28]:
np.random.rand(3)

array([0.3593185 , 0.30079546, 0.35850936])

### 3.2 Load fitting and testing result

In [29]:
# load fitting and testing result
fit_res = np.loadtxt('./fit_res.txt')
fit_res = fit_res.reshape(fit_res.shape[0], -1, 3)

print('fit result shape: ', fit_res.shape)
assert(fit_res.shape == sub_train_keypoints.shape)


fit result shape:  (42, 7, 3)


### 3.3 Compare fitting result with ground truth
- ground_truth: draw with smaller points
- estimated: draw with bigger points

In [30]:
def compare_fit_gt(idx):
    assert(idx < fit_res.shape[0])

    mesh_name = train_names[idx]
    vs, fs = load_mesh(mesh_name)

    gt = sub_train_keypoints[idx]
    est = fit_res[idx]
    
    #print('current mesh name: ', mesh_name)
    #print('keypoints diff: {}'.format(est - gt))
    compare_mesh_keypoints13(vs, fs, gt, est)


indices = np.arange(fit_res.shape[0])
for idx in indices:
    print('current idx: ', idx)
    compare_fit_gt(idx)

current idx:  0
current idx:  1
current idx:  2
current idx:  3


KeyboardInterrupt: 

### 3.4 Compare test result with ground truth

In [27]:
ratio = np.random.rand() * 0.2
print('ratio: ', ratio)
select = np.random.rand(2000)
select = select > ratio
print('#select: ', select.sum())

ratio:  0.006152294047235918
#select:  1987


In [19]:
select

array([ True,  True,  True,  True,  True])